In [61]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.dummy import DummyClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import make_classification
import os
import csv
import struct
import numpy as np
from scipy.interpolate import interp1d
from typing import List, Tuple
from numpy.core import ndarray
from dataclasses import dataclass
from sklearn.model_selection import LeaveOneOut
import math
from sklearn.utils import shuffle
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV

In [2]:
def class_name_to_numeric(cn: str) -> int:
    if cn == "SARS-CoV":
        return 0
    elif cn == "SARS-CoV-2":
        return 1
    elif cn == "MERS-CoV":
        return 2
    elif cn == "HCoV-229E":
        return 3
    else:
        return 4

def class_name_to_numeric_p_n(cn: str) -> int:
    if cn == "Positive":
        return 0
    elif cn == "Negative":
        return 1
    else:
        return 4

def numeric_to_class_name(cn: int) -> str:
    if cn == 0:
        return "SARS-CoV"
    elif cn == 1:
        return "SARS-CoV-2"
    elif cn == 2:
        return "MERS-CoV"
    elif cn == 3:
        return "HCoV-229E"
    else:
        return "Boh"

In [3]:
@dataclass
class Event:
    """Improved feature of single event"""
    class_name: str
    amplitude: float
    d10: float
    d20: float
    d30: float
    d40: float
    d50: float
    d60: float
    d70: float
    d80: float
    d90: float
    c10: float
    c20: float
    c30: float
    c40: float
    c50: float
    c60: float
    c70: float
    c80: float
    c90: float

def event_as_list_no_class_name(e: Event):
    return [e.amplitude, 
            e.d10,e.d20, e.d30, e.d40, e.d50, e.d60, e.d70, e.d80, e.d90,
            e.c10,e.c20, e.c30, e.c40, e.c50, e.c60, e.c70, e.c80, e.c90]

In [51]:
# desktop_folder = os.path.join("C:\\", "Users", "Luca Rossi", "Desktop")
desktop_folder = os.path.join("/home", "luca", "Desktop")
results_folder = os.path.join(desktop_folder, "RESULTS")
virus_folders = ["SARS-CoV", "SARS-CoV-2", "MERS-CoV", "HCoV-229E"]
random_state = 69

def open_dat(filename):
    f = open(filename, "rb")
    f_cont = f.read()
    f.close()
    raw = struct.unpack("d" * (len(f_cont) // 8), f_cont)
    return np.array(raw)

def extract_lengths(filename):
    with open(filename) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        line_count = 0
        events_lengths = []
        for row in csv_reader:
            if len(row) > 1:
                if line_count > 1 and len(row) == 2:
                    events_lengths.append(int(row[1]) - int(row[0]))
            line_count+=1
        return events_lengths

def extract_raw_events(dir_name) -> List[ndarray]:
    events = []
    files = os.listdir(dir_name)
    if len(files) == 0:
        print(dir_name)
        return
    dat_file = [os.path.join(dir_name, f) for f in files if f.endswith(".dat")].pop()
    details_file = [os.path.join(dir_name, f) for f in files if f.endswith(".csv")].pop()
    # caricamento eventi dal singolo file    
    loaded_events = open_dat(dat_file)
    # caricamento dettagli file
    events_length = extract_lengths(details_file)  
    b = 0
    for ev_len in events_length:
        e = b + ev_len
        event = np.array(loaded_events[b:e])
        b = e
        events.append(event)
    return events

def duration_x_and_center(event: ndarray, baseline, amplitude, percentage):
    event = event - baseline
    event = np.concatenate(([0],event,[0]))
    event_x = np.nonzero(event < amplitude * (1 - percentage) )[0]
    dx = event_x.size
    if dx % 2 != 0:
        return dx, event_x[dx//2]
    m1 = dx // 2
    m0 = m1 - 1
    return dx, (event_x[m1] + event_x[m0]) / 2 
    
def calc_baseline(event: ndarray) -> float:
    ev_len = event.size
    x_baseline = np.concatenate((event[:round(ev_len/5*0.2)], event[round(ev_len - ev_len/5*0.2):]))
    return np.mean(x_baseline)

def extract_events(raw_events: List[ndarray], class_name) -> Tuple[str, List[Event]]:
    if raw_events is None:
        return []
    events = []
    for raw_event in raw_events:
        baseline = calc_baseline(raw_event)
#         remove last first and last 2/5 which are padding
        raw_event = raw_event[raw_event.size // 5 * 2:raw_event.size // 5 * 3]
        peak = raw_event.max()
        amplitude = peak-baseline
        d10, c10 = duration_x_and_center(raw_event, baseline, amplitude, 0.1)
        d20, c20 = duration_x_and_center(raw_event, baseline, amplitude, 0.2)
        d30, c30 = duration_x_and_center(raw_event, baseline, amplitude, 0.3)
        d40, c40 = duration_x_and_center(raw_event, baseline, amplitude, 0.4)
        d50, c50 = duration_x_and_center(raw_event, baseline, amplitude, 0.5)
        d60, c60 = duration_x_and_center(raw_event, baseline, amplitude, 0.6)
        d70, c70 = duration_x_and_center(raw_event, baseline, amplitude, 0.7)
        d80, c80 = duration_x_and_center(raw_event, baseline, amplitude, 0.8)
        d90, c90 = duration_x_and_center(raw_event, baseline, amplitude, 0.9)
        events.append(Event(class_name, amplitude,
                              d10, d20, d30, d40, d50, d60, d70, d80, d90,
                              c10, c20, c30, c40, c50, c60, c70, c80, c90))
    return events
        
def get_classes_and_paths(results_folder: List[str], virus_folders:List[str]):
    class_and_path_to_virus_dir = [ (v, os.path.join(results_folder, v)) for v in virus_folders]
    classes_and_paths = []
    for v, p in class_and_path_to_virus_dir:
        for new_p in [os.path.join(p, d) for d in os.listdir(p)]:
            classes_and_paths.append((v, new_p))            
    return classes_and_paths

def predict_entire_file(clf, events: List[Event]):
    results = {"SARS-CoV": 0, "SARS-CoV-2": 0, "MERS-CoV": 0, "HCoV-229E": 0}
    features = [event_as_list_no_class_name(e) for e in events]
    predictions = clf.predict(features)
    results["SARS-CoV"] = len([p for p in predictions if p == 0])
    results["SARS-CoV-2"] = len([p for p in predictions if p == 1])
    results["MERS-CoV"] = len([p for p in predictions if p == 2])
    results["HCoV-229E"] = len([p for p in predictions if p == 3])
    print(results)
    return max(results, key=results.get)

def get_test_and_train(c_p_e, idx, l):
    test = []
    c_p_e_without_test = [c_p_e[i] for i in range(len(c_p_e)) if i != idx]
    train = []
    for c in virus_folders:
        train_with_class_c = [(ct,_,e) for ct,_,e in c_p_e_without_test if ct == c]
        events = [item for sublist in [e for _, _, e in train_with_class_c] for item in sublist]
        events = shuffle(events, random_state=random_state)
        train+=events[:l]
        test.append((c,events[l:]))
    assert(l * 4 == len(train))
    train = shuffle(train,random_state=random_state)
    return test, train
    

def predict_entire_file_p_n(clf, events: List[Event]):
    results = {"Positive": 0, "Negative": 0}
    features = [event_as_list_no_class_name(e) for e in events]
    predictions = clf.predict(features)
    results["Positive"] = len([p for p in predictions if p == "Positive"])
    results["Negative"] = len([p for p in predictions if p == "Negative"])
    print(results)
    return max(results, key=results.get)

def predict_entire_file_p_n_svc(clf, scaler, events: List[Event]):
    results = {"Positive": 0, "Negative": 0}
    features = [event_as_list_no_class_name(e) for e in events]
    features = scaler.transform(features)
    predictions = clf.predict_proba(features)
    print(predictions)
    results["Positive"] = len([p for p in predictions if p == "Positive"])
    results["Negative"] = len([p for p in predictions if p == "Negative"])
    print(results)
    return max(results, key=results.get)

In [5]:
c_p = get_classes_and_paths(results_folder, virus_folders)
c_p_re = [ (c, p, extract_raw_events(p)) for c, p in c_p]
c_p_e = [(c, p, extract_events(re, c)) for c, p, re in c_p_re]

In [6]:
confusion_matrix_with_balanced_training_data_and_leftovers = [[0,0,0,0],[0,0,0,0],[0,0,0,0],[0,0,0,0]]
# |test| = |train * 4|
confusion_matrix_balanced_training_data = [[0,0,0,0],[0,0,0,0],[0,0,0,0],[0,0,0,0]]
clf = RandomForestClassifier(class_weight="balanced_subsample", random_state=random_state)
for idx in range(len(c_p_e)):
    print("CICLO ", idx)
    test_class, _, test_events = c_p_e[idx]
    test, train = get_test_and_train(c_p_e, idx, len(test_events))
    classes = [class_name_to_numeric(e.class_name) for e in train]
    features = [event_as_list_no_class_name(e) for e in train]
    clf.fit(features, classes)
    print(test_class)
    prediction = predict_entire_file(clf, test_events)
    confusion_matrix_balanced_training_data[class_name_to_numeric(test_class)][class_name_to_numeric(prediction)] += 1    
    print("others")
    for tc,te in test:
        print(tc)
        prediction = predict_entire_file(clf, te)
        confusion_matrix_with_balanced_training_data_and_leftovers[class_name_to_numeric(tc)][class_name_to_numeric(prediction)] += 1
    print("----------------------")

CICLO  0
SARS-CoV
{'SARS-CoV': 72, 'SARS-CoV-2': 7, 'MERS-CoV': 6, 'HCoV-229E': 6}
others
SARS-CoV
{'SARS-CoV': 471, 'SARS-CoV-2': 98, 'MERS-CoV': 136, 'HCoV-229E': 134}
SARS-CoV-2
{'SARS-CoV': 148, 'SARS-CoV-2': 518, 'MERS-CoV': 89, 'HCoV-229E': 155}
MERS-CoV
{'SARS-CoV': 175, 'SARS-CoV-2': 88, 'MERS-CoV': 237, 'HCoV-229E': 134}
HCoV-229E
{'SARS-CoV': 281, 'SARS-CoV-2': 288, 'MERS-CoV': 335, 'HCoV-229E': 806}
----------------------
CICLO  1
SARS-CoV
{'SARS-CoV': 93, 'SARS-CoV-2': 55, 'MERS-CoV': 65, 'HCoV-229E': 39}
others
SARS-CoV
{'SARS-CoV': 297, 'SARS-CoV-2': 54, 'MERS-CoV': 106, 'HCoV-229E': 60}
SARS-CoV-2
{'SARS-CoV': 82, 'SARS-CoV-2': 468, 'MERS-CoV': 88, 'HCoV-229E': 111}
MERS-CoV
{'SARS-CoV': 95, 'SARS-CoV-2': 77, 'MERS-CoV': 202, 'HCoV-229E': 99}
HCoV-229E
{'SARS-CoV': 216, 'SARS-CoV-2': 269, 'MERS-CoV': 240, 'HCoV-229E': 824}
----------------------
CICLO  2
SARS-CoV
{'SARS-CoV': 149, 'SARS-CoV-2': 8, 'MERS-CoV': 30, 'HCoV-229E': 25}
others
SARS-CoV
{'SARS-CoV': 337, 'SARS-C

HCoV-229E
{'SARS-CoV': 43, 'SARS-CoV-2': 36, 'MERS-CoV': 36, 'HCoV-229E': 148}
others
SARS-CoV
{'SARS-CoV': 412, 'SARS-CoV-2': 75, 'MERS-CoV': 162, 'HCoV-229E': 109}
SARS-CoV-2
{'SARS-CoV': 114, 'SARS-CoV-2': 430, 'MERS-CoV': 89, 'HCoV-229E': 105}
MERS-CoV
{'SARS-CoV': 114, 'SARS-CoV-2': 69, 'MERS-CoV': 178, 'HCoV-229E': 101}
HCoV-229E
{'SARS-CoV': 188, 'SARS-CoV-2': 235, 'MERS-CoV': 214, 'HCoV-229E': 638}
----------------------
CICLO  20
HCoV-229E
{'SARS-CoV': 48, 'SARS-CoV-2': 56, 'MERS-CoV': 45, 'HCoV-229E': 250}
others
SARS-CoV
{'SARS-CoV': 353, 'SARS-CoV-2': 82, 'MERS-CoV': 108, 'HCoV-229E': 79}
SARS-CoV-2
{'SARS-CoV': 84, 'SARS-CoV-2': 371, 'MERS-CoV': 71, 'HCoV-229E': 76}
MERS-CoV
{'SARS-CoV': 80, 'SARS-CoV-2': 54, 'MERS-CoV': 134, 'HCoV-229E': 58}
HCoV-229E
{'SARS-CoV': 132, 'SARS-CoV-2': 195, 'MERS-CoV': 192, 'HCoV-229E': 484}
----------------------
CICLO  21
HCoV-229E
{'SARS-CoV': 10, 'SARS-CoV-2': 17, 'MERS-CoV': 8, 'HCoV-229E': 9}
others
SARS-CoV
{'SARS-CoV': 519, 'SARS-CoV

In [7]:
print(np.array(confusion_matrix_balanced_training_data))
print(np.array(confusion_matrix_with_balanced_training_data_and_leftovers))

[[5 0 0 0]
 [0 3 0 0]
 [3 0 3 0]
 [1 2 2 7]]
[[26  0  0  0]
 [ 0 25  0  1]
 [ 2  0 24  0]
 [ 0  1  0 25]]


# Saliva

In [8]:
training_folder = os.path.join(desktop_folder, "TRAINING")
test_folder = os.path.join(desktop_folder, "TEST")
positive_negative_folders =["Positive", "Negative"]

# Extract training data and train model

In [9]:
c_p_saliva_train = get_classes_and_paths(training_folder, positive_negative_folders)
c_p_re_saliva_train = [ (c, p, extract_raw_events(p)) for c, p in c_p_saliva_train]
c_p_e_saliva_train = [(c, p, extract_events(re, c)) for c, p, re in c_p_re_saliva_train]
print(len(c_p_e_saliva_train))
print([p for _, p, e in c_p_e_saliva_train if len(e)==0])
c_p_e_saliva_train = [(c, p, e) for c, p, e in c_p_e_saliva_train if len(e)>0]
print(len(c_p_e_saliva_train))


c_p_saliva_test = get_classes_and_paths(test_folder, positive_negative_folders)
c_p_re_saliva_test = [ (c, p, extract_raw_events(p)) for c, p in c_p_saliva_test]
c_p_e_saliva_test = [(c, p, extract_events(re, c)) for c, p, re in c_p_re_saliva_test]
print(len(c_p_e_saliva_test))
c_p_e_saliva_test = [(c, p, e) for c, p, e in c_p_e_saliva_test if len(e)>0]
print(len(c_p_e_saliva_test))


/home/luca/Desktop/TRAINING/Positive/F8 day1
/home/luca/Desktop/TRAINING/Positive/AS-2-2-bias+01_BK-953_045fil_TI_1st
/home/luca/Desktop/TRAINING/Positive/AS-2-2-bias+01_BK-1126_045fil_TI
/home/luca/Desktop/TRAINING/Positive/F2
/home/luca/Desktop/TRAINING/Negative/HD-120420-17
/home/luca/Desktop/TRAINING/Negative/HD-112720-58
/home/luca/Desktop/TRAINING/Negative/HD-120720-6
/home/luca/Desktop/TRAINING/Negative/HD-120420-33
/home/luca/Desktop/TRAINING/Negative/HD-120420-44
/home/luca/Desktop/TRAINING/Negative/HD-112720-46
/home/luca/Desktop/TRAINING/Negative/HD-120720-46
/home/luca/Desktop/TRAINING/Negative/HD-120420-32
80
['/home/luca/Desktop/TRAINING/Positive/F8 day1', '/home/luca/Desktop/TRAINING/Positive/AS-2-2-bias+01_BK-953_045fil_TI_1st', '/home/luca/Desktop/TRAINING/Positive/AS-2-2-bias+01_BK-1126_045fil_TI', '/home/luca/Desktop/TRAINING/Positive/F2', '/home/luca/Desktop/TRAINING/Negative/HD-120420-17', '/home/luca/Desktop/TRAINING/Negative/HD-112720-58', '/home/luca/Desktop/TRA

In [46]:
positive_events = [item for sublist in [e for ct,_,e in c_p_e_saliva_train if ct == "Positive"] for item in sublist]
negetive_events = [item for sublist in [e for ct,_,e in c_p_e_saliva_train if ct == "Negative"] for item in sublist]
print(len(positive_events))
print(len(negetive_events))

15034
9336


In [47]:
positive_events = shuffle(positive_events, random_state=random_state)
# get only subset of positive events to make it more balanced
positive_events = positive_events[:len(negetive_events)]
print(len(positive_events))
print(len(negetive_events))

9336
9336


In [48]:
train = positive_events+negetive_events
train = shuffle(train, random_state=random_state)
train_labels = [e.class_name for e in train]
train_X = [event_as_list_no_class_name(e) for e in train]
train_X = np.array(train_X)

## Train classifier

In [13]:
from sklearn.model_selection import RandomizedSearchCV
from pprint import pprint
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]

# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

pprint(random_grid)

{'bootstrap': [True, False],
 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]}


In [14]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestClassifier(random_state = 69)
# Random search of parameters, using 5 fold cross validation, 
# search across 100 different combinations, and use all available cores
cv = train_X.size
rf_random = RandomizedSearchCV(estimator=rf, param_distributions=random_grid,
                              n_iter = 100, cv = 10, verbose=2, random_state=69, 
                              n_jobs=-1, return_train_score=True)

# Fit the random search model
rf_random.fit(train_X, train_labels)

Fitting 10 folds for each of 100 candidates, totalling 1000 fits


RandomizedSearchCV(cv=10, estimator=RandomForestClassifier(random_state=69),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [10, 20, 30, 40, 50, 60,
                                                      70, 80, 90, 100, 110,
                                                      None],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [200, 400, 600, 800,
                                                         1000, 1200, 1400, 1600,
                                                         1800, 2000]},
                   random_state=69, return_train_score=True, verbose=2)

In [15]:
rf_random.best_params_

{'n_estimators': 800,
 'min_samples_split': 5,
 'min_samples_leaf': 1,
 'max_depth': 10,
 'bootstrap': False}

In [21]:
saliva_classifier = RandomForestClassifier(random_state=random_state, n_estimators=800, min_samples_split=5, min_samples_leaf=1,max_depth=10,bootstrap=True, oob_score=True)
saliva_classifier.fit(train_X, train_labels)
print(saliva_classifier.oob_score_)

0.5318658954584404


## Test classifier

In [19]:
saliva_confusion_matrix = [[0,0],[0,0]]
for tc,_,te in c_p_e_saliva_test:
    print(tc)
    prediction = predict_entire_file_p_n(saliva_classifier, te)
    saliva_confusion_matrix[class_name_to_numeric_p_n(tc)][class_name_to_numeric_p_n(prediction)] += 1

Positive
{'Positive': 703, 'Negative': 508}
Positive
{'Positive': 589, 'Negative': 320}
Positive
{'Positive': 25, 'Negative': 24}
Positive
{'Positive': 161, 'Negative': 205}
Positive
{'Positive': 58, 'Negative': 206}
Positive
{'Positive': 421, 'Negative': 772}
Positive
{'Positive': 52, 'Negative': 60}
Positive
{'Positive': 524, 'Negative': 482}
Positive
{'Positive': 92, 'Negative': 125}
Positive
{'Positive': 227, 'Negative': 213}
Positive
{'Positive': 111, 'Negative': 199}
Positive
{'Positive': 79, 'Negative': 104}
Positive
{'Positive': 137, 'Negative': 283}
Positive
{'Positive': 91, 'Negative': 99}
Positive
{'Positive': 111, 'Negative': 54}
Positive
{'Positive': 82, 'Negative': 87}
Positive
{'Positive': 63, 'Negative': 160}
Positive
{'Positive': 835, 'Negative': 541}
Positive
{'Positive': 150, 'Negative': 142}
Positive
{'Positive': 145, 'Negative': 71}
Positive
{'Positive': 42, 'Negative': 48}
Positive
{'Positive': 172, 'Negative': 116}
Positive
{'Positive': 22, 'Negative': 31}
Positi

In [18]:
np.array(saliva_confusion_matrix)

array([[19, 23],
       [28, 19]])

# Testing out another method to ensemble results and using a different classifier

# Preprocessing data: Standardization

In [49]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(train_X)
std_train_X = scaler.transform(train_X)
# REMEMBER TO PREPROCESS TEST DATA TOO

## Grid search for svm

In [53]:
kernel = ["linear", "poly", "rbf", "sigmoid", "precomputed"]
degree = [1,3,5]

random_grid_svc = {'kernel': kernel,
                   'degree': degree}

In [ ]:
cv = train_X.size
svc = SVC(probability=True, random_state=random_state)
svc_random = GridSearchCV(svc, random_grid_svc)

# Fit the random search model
svc_random.fit(train_X, train_labels)

## Train SVM

In [50]:
from sklearn.svm import SVC
svc = SVC(probability=True, random_state=random_state)
svc.fit(std_train_X, train_labels)

SVC(probability=True)

## Test svm

In [52]:
saliva_confusion_matrix_svc = [[0,0],[0,0]]
for tc,_,te in c_p_e_saliva_test:
    print(tc)
    prediction = predict_entire_file_p_n_svc(svc, scaler, te)
    saliva_confusion_matrix_svc[class_name_to_numeric_p_n(tc)][class_name_to_numeric_p_n(prediction)] += 1

Positive
[[0.52462501 0.47537499]
 [0.5469997  0.4530003 ]
 [0.52020325 0.47979675]
 [0.58986281 0.41013719]
 [0.51870564 0.48129436]
 [0.50640184 0.49359816]
 [0.52552689 0.47447311]
 [0.4786501  0.5213499 ]
 [0.5648397  0.4351603 ]
 [0.403517   0.596483  ]
 [0.52972918 0.47027082]
 [0.47763744 0.52236256]
 [0.45824897 0.54175103]
 [0.48372847 0.51627153]
 [0.46198477 0.53801523]
 [0.56542558 0.43457442]
 [0.4033154  0.5966846 ]
 [0.51201729 0.48798271]
 [0.4704788  0.5295212 ]
 [0.42571893 0.57428107]
 [0.53486406 0.46513594]
 [0.49386702 0.50613298]
 [0.56108371 0.43891629]
 [0.46862961 0.53137039]
 [0.47317743 0.52682257]
 [0.45441746 0.54558254]
 [0.4706837  0.5293163 ]
 [0.44546211 0.55453789]
 [0.42320888 0.57679112]
 [0.56419006 0.43580994]
 [0.52672987 0.47327013]
 [0.55671624 0.44328376]
 [0.45636589 0.54363411]
 [0.48429211 0.51570789]
 [0.5        0.5       ]
 [0.46571311 0.53428689]
 [0.50581894 0.49418106]
 [0.5        0.5       ]
 [0.48078374 0.51921626]
 [0.51045059 0.4

/tmp/ipykernel_8449/1463982065.py:135: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  results["Positive"] = len([p for p in predictions if p == "Positive"])
/tmp/ipykernel_8449/1463982065.py:136: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  results["Negative"] = len([p for p in predictions if p == "Negative"])


[[0.40914867 0.59085133]
 [0.51486989 0.48513011]
 [0.43533995 0.56466005]
 ...
 [0.43986292 0.56013708]
 [0.46255703 0.53744297]
 [0.51920851 0.48079149]]
{'Positive': 0, 'Negative': 0}
Positive
[[0.55711535 0.44288465]
 [0.56639628 0.43360372]
 [0.58864327 0.41135673]
 [0.54792101 0.45207899]
 [0.52746613 0.47253387]
 [0.58308469 0.41691531]
 [0.56872112 0.43127888]
 [0.53842217 0.46157783]
 [0.5482303  0.4517697 ]
 [0.58378781 0.41621219]
 [0.5835221  0.4164779 ]
 [0.48716659 0.51283341]
 [0.54111636 0.45888364]
 [0.52830651 0.47169349]
 [0.54289986 0.45710014]
 [0.55572294 0.44427706]
 [0.5640804  0.4359196 ]
 [0.52065352 0.47934648]
 [0.54067208 0.45932792]
 [0.57698363 0.42301637]
 [0.51060079 0.48939921]
 [0.51622523 0.48377477]
 [0.52160642 0.47839358]
 [0.5505478  0.4494522 ]
 [0.58803159 0.41196841]
 [0.51359061 0.48640939]
 [0.5        0.5       ]
 [0.5985946  0.4014054 ]
 [0.55931318 0.44068682]
 [0.52200086 0.47799914]
 [0.45576666 0.54423334]
 [0.58113019 0.41886981]
 [0.

/tmp/ipykernel_8449/1463982065.py:135: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  results["Positive"] = len([p for p in predictions if p == "Positive"])
/tmp/ipykernel_8449/1463982065.py:136: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  results["Negative"] = len([p for p in predictions if p == "Negative"])
/tmp/ipykernel_8449/1463982065.py:135: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  results["Positive"] = len([p for p in predictions if p == "Positive"])
/tmp/ipykernel_8449/1463982065.py:136: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  results["Negative"] = len([p for p in predictions if p == "Negative"])


[[0.52351872 0.47648128]
 [0.4245996  0.5754004 ]
 [0.47679611 0.52320389]
 ...
 [0.5297531  0.4702469 ]
 [0.49408171 0.50591829]
 [0.53446226 0.46553774]]
{'Positive': 0, 'Negative': 0}
Positive
[[0.55455006 0.44544994]
 [0.4218108  0.5781892 ]
 [0.5        0.5       ]
 [0.51253632 0.48746368]
 [0.5        0.5       ]
 [0.5252437  0.4747563 ]
 [0.55682269 0.44317731]
 [0.54376535 0.45623465]
 [0.51188995 0.48811005]
 [0.51369431 0.48630569]
 [0.5        0.5       ]
 [0.51602293 0.48397707]
 [0.51637045 0.48362955]
 [0.57415557 0.42584443]
 [0.5        0.5       ]
 [0.43017969 0.56982031]
 [0.50769655 0.49230345]
 [0.57129435 0.42870565]
 [0.44946763 0.55053237]
 [0.45797128 0.54202872]
 [0.53670663 0.46329337]
 [0.45053461 0.54946539]
 [0.55421093 0.44578907]
 [0.53330405 0.46669595]
 [0.49106025 0.50893975]
 [0.56989378 0.43010622]
 [0.52261926 0.47738074]
 [0.53925357 0.46074643]
 [0.40322292 0.59677708]
 [0.53842979 0.46157021]
 [0.53026414 0.46973586]
 [0.4625666  0.5374334 ]
 [0.

...

/tmp/ipykernel_8449/1463982065.py:135: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  results["Positive"] = len([p for p in predictions if p == "Positive"])
/tmp/ipykernel_8449/1463982065.py:136: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  results["Negative"] = len([p for p in predictions if p == "Negative"])


[[0.47793896 0.52206104]
 [0.47810219 0.52189781]
 [0.43001408 0.56998592]
 ...
 [0.46736689 0.53263311]
 [0.42810097 0.57189903]
 [0.45633743 0.54366257]]
{'Positive': 0, 'Negative': 0}
Negative
[[0.5        0.5       ]
 [0.54919771 0.45080229]
 [0.56027972 0.43972028]
 [0.56000751 0.43999249]
 [0.5399723  0.4600277 ]
 [0.54546266 0.45453734]
 [0.5731454  0.4268546 ]
 [0.55931788 0.44068212]
 [0.54663442 0.45336558]
 [0.53412278 0.46587722]
 [0.53859779 0.46140221]
 [0.5        0.5       ]
 [0.5450736  0.4549264 ]
 [0.55614562 0.44385438]
 [0.5        0.5       ]
 [0.53687753 0.46312247]
 [0.54389542 0.45610458]
 [0.56066206 0.43933794]
 [0.54258462 0.45741538]
 [0.51068765 0.48931235]
 [0.59569971 0.40430029]
 [0.52661287 0.47338713]
 [0.51198958 0.48801042]
 [0.55822438 0.44177562]
 [0.45537933 0.54462067]
 [0.56260312 0.43739688]
 [0.53070243 0.46929757]
 [0.56370563 0.43629437]
 [0.5193565  0.4806435 ]
 [0.55819882 0.44180118]
 [0.44790888 0.55209112]
 [0.54560198 0.45439802]
 [0.

/tmp/ipykernel_8449/1463982065.py:135: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  results["Positive"] = len([p for p in predictions if p == "Positive"])
/tmp/ipykernel_8449/1463982065.py:136: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  results["Negative"] = len([p for p in predictions if p == "Negative"])
/tmp/ipykernel_8449/1463982065.py:135: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  results["Positive"] = len([p for p in predictions if p == "Positive"])
/tmp/ipykernel_8449/1463982065.py:136: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  results["Negative"] = len([p for p in predictions if p == "Negative"])


[[0.46980883 0.53019117]
 [0.54093891 0.45906109]
 [0.48846984 0.51153016]
 ...
 [0.46245515 0.53754485]
 [0.42585478 0.57414522]
 [0.39559474 0.60440526]]
{'Positive': 0, 'Negative': 0}


/tmp/ipykernel_8449/1463982065.py:135: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  results["Positive"] = len([p for p in predictions if p == "Positive"])
/tmp/ipykernel_8449/1463982065.py:136: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  results["Negative"] = len([p for p in predictions if p == "Negative"])


In [45]:
np.array(saliva_confusion_matrix_svc)

array([[42,  0],
       [47,  0]])